In [ ]:
import jieba
import codecs
from scipy.misc import imread
import os 
from os import path
import matpltlib.pyplot as plt
from PIL import Image, ImageDraw,ImageFont
from wordcloud import WordCloud, ImageColorGenerator
import DrawBarChart

In [1]:
if _name_ =='_main_':
    doc=''
    seged_list=segment(doc)   #對DOC進行斷字
    words_clear=removeStopWords(seged_list)  #萃取出有意義的詞彙
    drawWordCloud(words_clear)   #繪製文字雲
    wordCount(words_clear)   #對各詞彙進行出現頻率計算
    DrawBarChart.drawBar()   #繪製長條圖

IndentationError: expected an indented block (1580863088.py, line 1)

In [ ]:
def segment(doc):
    #以精準模式，將doc內的文字內容進行斷字，且所有取出的詞彙將逐一的，以一個字元的空白間隔，連接成字串
    seg_list=''.join(jieba.cut(doc,cut_all=False))
    
    #將斷詞結果存入result.txt中
    document_seged=codecs.open('result.txt','w+','utf-8')
    document_seged.write(seg_list)
    document_seged.close()
    
    return seg_list

In [ ]:
def removeStopWords(seged_list):
    words_stop_removed=[]
    
    #開啟預先準備好的stopword（停詞）檔案，stopword代表未來不會被列入統計的詞彙，也就是要淘汰的詞彙。
    stop_words=open(r'my_stopwords.txt')
    stop_words_text=stop_words.read()
    stop_words.close()
    
    #以'\n'為間隔，逐一抽離各停詞，並成為stop_words_list串列的元素
    stop_words_list=stop_words_text.split('\n')
    
    #再以‘’符號為間隔，逐一抽離各詞彙，並成為seged_text_list串列的元素
    seged_text_list=seged_list.split('')
    
    #萃取出未來需要進行統計的詞彙
    for word in seged_text_list:
        if word not in stop_words_list:
            words_stop_removed.append(word)
            
    #將萃取出的詞彙，存入result_stop.txt檔中
    words_clear=codecs.open('result_stop.txt','w','utf-8')
    words_clear.write(''.join(words_stop_removed))
    return ''.join(words_stop_removed)

In [ ]:
def drawWordCloud(seged_list):
    #以imread方法匯入圖形，用於設置文字雲的形狀，亦即設定遮罩的意思
    #設定遮罩的圖形為台灣地圖（taiwan_map.png）
    fig_mask=imread(r'taiwan_map.png')
    
    #產生文字雲物件，並設定其各種屬性
    wc=WordCloud(
       font_path=r'BMAO00HU.ttf',    #設定字型，需含路徑
       background_color='white',     #設定背景顏色
       mask='fig_mask',              #設定遮罩
       max_words=2000,               #設定文字雲能顯示的最大詞數
       max_font_size=60              #設定最大字型
    )
    
    #使用seged_list的詞彙製作文字雲
    wc.generate(seged_list)
    #儲存文字雲圖
    wc.to_file('wc_fig.jpg')
    
    #畫第一張文字雲圖，字體顏色自動產生
    plt.imshow(wc,interpolation='bilinear')
    plt.axis('off') #不顯示座標軸
    
    #畫第二張圖，沒有使用plt.figure()的話，則第二張圖會把第一張圖蓋掉
    plt.figure()
    
    #設定字體顏色，將依照遮罩圖內的顏色分佈來做配色
    image_colors=ImageColorGeneraor(fig_mask)
    
    #第二張文字雲圖中，改變字體顏色，color_func代表產生新顏色，新顏色為依照遮罩圖內原始的顏色分佈來做配色
    plt.imshow(wc.recolor(color_func=image_colors),interpolation='bilinear')
    plt.axis('off')
    
    #把文字雲圖顯示出來
    plt.show()
    

In [ ]:
def wordCount(seged_list):
    word_lst=[]
    word_dict=[]
    with codecs.open('word_Count.txt','w','utf-8')as wcf:
        
        #以一個字元的空白間隔，逐一抽離seged_list中的各詞彙，並成為word_lst串列中的元素
        word_lst.append(seged_list.split(''))
        
        #遍歷所有的詞彙
        for item in word_lst:
            #計算各詞彙出現的次數
            for item2 in item:
                if len(item2)>=2:
                    if item2 not in word_dict:
                        word_dic[item2]=1
                    else:
                        word_dic[item2]+=1
                        
        #字典中，以值為key，進行各詞彙元素的遞減排序，排序完成後，存入word_dict_sorted字典
        word_dict_sorted=dic(sorted(word_dict.items(),key=lambda item:item[1],reverse=True))
        
        #遍歷word_dict_sorted字典中的所有的詞彙元素，並轉為字串鍵值對而存於word_Count.txt中
        for key in word_dict_sorted:
            wcf.write(key+''+str(word_dict_sorted[key])+'\n')
            
    wcf.close()